#### Train the Model**

We used a simple yet powerful Random Forest Classifier to identify speakers based on their voice features.

In [25]:
# Import libraries

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import joblib
import os

#### **Load processed audio CSV**

In [35]:
csv_path = r"C:\users\LENOVO\Documents\formative2-mlp\Audios\processed_audio_features.csv"
df = pd.read_csv(csv_path)
print(f" Loaded data: {df.shape[0]} rows, {df.shape[1]} columns")
print(" Audio features loaded successfully!")
print(df.head())

 Loaded data: 32 rows, 69 columns
 Audio features loaded successfully!
                                sample_id  person  phrase       augmentation  \
0      Egide_Confirm_Transaction_original     NaN     NaN           Original   
1      Egide_Confirm_Transaction_pitch_up     NaN     NaN     Pitch Shift +2   
2  Egide_Confirm_Transaction_time_stretch     NaN     NaN  Time Stretch 1.1x   
3         Egide_Confirm_Transaction_noise     NaN     NaN   Background Noise   
4              Egide_Yes_Approve_original     NaN     NaN           Original   

   sample_rate  mfcc1_mean  mfcc1_std  mfcc1_min  mfcc1_max  mfcc2_mean  ...  \
0        44100  -357.26420   78.29712 -565.46690 -219.59917  152.541720  ...   
1        44100  -382.09753   78.63037 -559.86890 -256.65927  137.081760  ...   
2        44100  -390.32140   75.74882 -571.45750 -270.21460  147.054170  ...   
3        44100  -158.54604   40.17845 -269.86728  -75.82134   63.481052  ...   
4        44100  -396.49390   83.95285 -559.94037

#### **Fill missing 'person' from 'sample_id'**

In [26]:
if df['person'].isna().all():
    print(" 'person' column empty. Extracting from 'sample_id'...")
    df['person'] = df['sample_id'].apply(lambda x: x.split('_')[0])

# Drop rows with missing 'person'
df = df.dropna(subset=['person'])
print(f" Data after filling 'person': {df.shape[0]} rows")

 'person' column empty. Extracting from 'sample_id'...
 Data after filling 'person': 32 rows


#### **Prepare features (X) and labels (y)**

In [28]:
exclude_cols = ['sample_id', 'person', 'phrase', 'augmentation', 'sample_rate']
X = df.drop(columns=[col for col in exclude_cols if col in df.columns])
y = df['person']

print(f"Features shape: {X.shape}")
print(f"Number of classes: {len(y.unique())}")

Features shape: (32, 64)
Number of classes: 4


#### **Split Data for Training and Testing**

We divide the dataset into training (80%) and testing (20%). This helps evaluate how well your model generalizes.

In [29]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Training samples: {X_train.shape[0]}, Testing samples: {X_test.shape[0]}")

Training samples: 25, Testing samples: 7


#### **Train Random Forest Classifier**

In [30]:
model = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    max_depth=None,
    n_jobs=-1
)

model.fit(X_train, y_train)
print(" Model training complete!")

 Model training complete!


#### **Evaluate on test set**

In [31]:
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Test accuracy: {acc*100:.2f}%")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Test accuracy: 71.43%

Classification Report:
              precision    recall  f1-score   support

       Egide       0.50      1.00      0.67         2
          JD       1.00      1.00      1.00         1
      Mariam       0.00      0.00      0.00         2
      Noella       1.00      1.00      1.00         2

    accuracy                           0.71         7
   macro avg       0.62      0.75      0.67         7
weighted avg       0.57      0.71      0.62         7



C:\Users\LENOVO\anaconda3\New folder\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\LENOVO\anaconda3\New folder\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\LENOVO\anaconda3\New folder\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### **Step 8: Save model and metadata**

In [32]:
save_dir = r"C:\users\LENOVO\Documents\formative2-mlp\Audios"
os.makedirs(save_dir, exist_ok=True)

joblib.dump(model, os.path.join(save_dir, 'audio_model.pkl'))
joblib.dump(X.columns.tolist(), os.path.join(save_dir, 'feature_names.pkl'))
joblib.dump(model.classes_, os.path.join(save_dir, 'class_names.pkl'))

print(f" Model and metadata saved in {save_dir}")

 Model and metadata saved in C:\users\LENOVO\Documents\formative2-mlp\Audios
